In [1]:
import nltk
import collections
import warnings
from sklearn.pipeline import Pipeline
import pandas as pd
import seaborn as sns
from definitions import ROOT_DIR
nltk.download('punkt')

import string
from src.preprocessing.custom_transformers import PunctuationRemover, StopWordsRemover, IntoLowerCase, ShortToLong
from src.visualizers.visualiers import get_stats, plot_counter, count_words

ModuleNotFoundError: No module named 'definitions'

In [ ]:
# Download Quora database
q_df = pd.read_csv(ROOT_DIR+'/raw_data/quora_dataset.csv')
print(q_df.loc[q_df['target']==1].head(3))

In [ ]:
# Replace dataset specific symbols to common ones
q_df['question_text'] = q_df['question_text'].str.replace("’", "'")

In [ ]:
# Target value distribution
sns.set(style="darkgrid")
sns.countplot(x='target', data=q_df)

In [ ]:
# Add column with number of words
q_df['num_words'] = q_df['question_text'].apply(
    lambda x: len([token for token in x.split(" ") if token != ""])
)

In [ ]:
# Print stats for number of words
print("Stats for whole dataset:")
get_stats(q_df, 'words', 'num_words')

In [ ]:
# One word as a question? It is interesting to look into.
q_df.loc[q_df['num_words']==1, ]

It seems that all one question are considered as trolling. This is a questionable approach of those who labeled the data. But for this specific case I will agree with it.

In [ ]:
# Stats for number of words if target value is 0
q_df_0 = q_df.copy().loc[q_df['target']==0, ]
print("Stats for 0 target values:")
get_stats(q_df_0, 'words', 'num_words')

In [ ]:
# Stats for number of words if target value is 1
q_df_1 = q_df.copy().loc[q_df['target']==1, ]
print("Stats for 0 target values:")
get_stats(q_df_1, 'words', 'num_words')

It seems like authors of troll questions are too busy to type more than 64 words.vAlso distribution of number of words shows spikes with some ordered pattern. My suggestion is that trolling questions were automatically created (either for pupose of this competition or by Quora's trolls, when they posted questions). The conclusion from this section of analysis: max of 40 words is enough for padding.

In [ ]:
# Download stop words from nltk library
nltk.download('stopwords')
eng_stop_words = nltk.corpus.stopwords.words('english') 

In [ ]:
# Add column for text transformations
q_df['prep_text'] = q_df['question_text'].copy()

In [ ]:
pipeline = Pipeline(
    steps=[
        ('contracted', ShortToLong(['prep_text'])),
        ('punctuation', PunctuationRemover(['prep_text'])),
        ('lowercase', IntoLowerCase(['prep_text'])),
        ('stopwords', StopWordsRemover(['prep_text'], eng_stop_words))
    ]
)

In [ ]:
# Transform the dataset using created pipeline
pipeline.fit_transform(q_df)
q_df.to_csv(ROOT_DIR + '/temp_data/quora_df_no_stop_words.csv', index=False)

In [ ]:
# Count unigrams frequency for whole dataset
count_words(q_df, 'prep_text', 'unigrams', 15, ' for whole dataset')

In [ ]:
# Count bigrams frequency for whole dataset
count_words(q_df, 'prep_text', 'bigrams', 15, ' for whole dataset')

In [ ]:
# Count words frequency for sinsere questions only
count_words(q_df.loc[q_df['target'] == 0, ], 'prep_text', 'unigrams', 15, ' for sinsere questions only')

In [ ]:
# Count words frequency for sinsere questions only
count_words(q_df.loc[q_df['target'] == 0, ], 'prep_text', 'bigrams', 15, ' for sinsere questions only')

In [ ]:
# Count words frequency for sinsere questions only
count_words(q_df.loc[q_df['target'] == 0, ], 'prep_text', 'trigrams', 15, ' for sinsere questions only')

In [ ]:
# Count words frequency for insinsere questions only
count_words(q_df.loc[q_df['target'] == 1, ], 'prep_text', 'unigrams', 15, ' for insinsere questions only')

In [ ]:
# Count words frequency for insinsere questions only
count_words(q_df.loc[q_df['target'] == 1, ], 'prep_text', 'bigrams', 15, ' for insinsere questions only')

In [ ]:
# Count words frequency for insinsere questions only
count_words(q_df.loc[q_df['target'] == 1, ], 'prep_text', 'trigrams', 15, ' for insinsere questions only')